In [2]:
!nvidia-smi

Sun Oct  1 17:41:17 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 537.42                 Driver Version: 537.42       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4080      WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   38C    P8              11W / 320W |   1334MiB / 16376MiB |     15%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [7]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/d9/92/2d3aecf9f4a192968035880be3e2fc8b48d541c7128f7c936f430d6f96da/accelerate-0.23.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/258.1 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/258.1 kB 660.6 kB/s eta 0:00:01
   ---------- ---------------------------- 71.7/258.1 kB 975.2 kB/s eta 0:00:01
   ---------------------------------------  256.0/258.1 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 258.1/258.1 kB 2.0 MB/s eta 0:00:00
Found existing installation: transformers 4.33.3
Uninstalling transformers-4.33.3:
  Successfully uninstalled transformers-4.33.3
Found existing installation: accelerate 0.23.0
Uninstalling accelerate-0.23.0:
  Successfully uninstalled accelerate-0.23.0
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/98/46/f6a79f944d5c7763a9bc13b2aa6ac72daf43a6551f5fb0

In [4]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Owner\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [7]:
model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
!pip install wget

In [9]:
import wget
import zipfile

# Define the URL of the file to download
url = "https://github.com/Shekhar-Khairnar/text-summarizer/raw/main/dataset/samsumdata.zip"

# Define the destination directory where you want to save the downloaded file
destination_dir = r"C:\Users\Owner\Downloads\github\text-summarizer\dataset"

# Download the file using wget
wget.download(url, destination_dir)

# Unzip the downloaded file
with zipfile.ZipFile(destination_dir + "/samsumdata.zip", 'r') as zip_ref:
    zip_ref.extractall(destination_dir)

# Confirm that the file has been downloaded and unzipped successfully
print("File downloaded and unzipped successfully.")

HTTPError: HTTP Error 404: Not Found

In [11]:
dataset_samsum = load_from_disk(r'C:\Users\Owner\Downloads\dataset\samsumdata\samsum_dataset')
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [12]:

split_lengths = [len(dataset_samsum[split])for split in dataset_samsum]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue:")

print(dataset_samsum["test"][1]["dialogue"])

print("\nSummary:")

print(dataset_samsum["test"][1]["summary"])

Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


In [13]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
    
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )
        
    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }
    

In [14]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/819 [00:00<?, ? examples/s]c:\Users\Owner\anaconda3\envs\textsummarizer\lib\site-packages\transformers\tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 819/819 [00:00<00:00, 8077.86 examples/s]


In [15]:
dataset_samsum_pt["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [16]:
# Training

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [17]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=10, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
) 

In [18]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"], #just to check the model performance test set is passed
                  eval_dataset=dataset_samsum_pt["validation"])

In [19]:
trainer.train()

  0%|          | 10/9200 [00:13<2:53:15,  1.13s/it]

{'loss': 2.8809, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}


  0%|          | 20/9200 [00:24<2:54:03,  1.14s/it]

{'loss': 3.0973, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.02}


  0%|          | 30/9200 [00:36<2:57:43,  1.16s/it]

...

{'loss': 1.4149, 'learning_rate': 4.1379310344827587e-05, 'epoch': 2.17}


                                                     
 22%|██▏       | 2000/9200 [39:54<2:52:02,  1.43s/it]

{'eval_loss': 1.3599623441696167, 'eval_runtime': 23.5676, 'eval_samples_per_second': 34.709, 'eval_steps_per_second': 34.709, 'epoch': 2.17}


 22%|██▏       | 2001/9200 [39:56<16:59:46,  8.50s/it]

KeyboardInterrupt: 

In [35]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, 
                               batch_size=16, device=device, 
                               column_text="article", 
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
        
        inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                        padding="max_length", return_tensors="pt")
        
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device), 
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
        
        # Finally, we decode the generated texts, 
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                clean_up_tokenization_spaces=True) 
               for s in summaries]      
        
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
        
        
        metric.add_batch(predictions=decoded_summaries, references=target_batch)
        
    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [36]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

C:\Users\Owner\AppData\Local\Temp\ipykernel_14396\2696170338.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')


In [37]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

100%|██████████| 5/5 [00:13<00:00,  2.78s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.021832,0.0,0.021834,0.021788


In [39]:
## Save model
model_pegasus.save_pretrained(r"C:\Users\Owner\Downloads\github\text-summarizer\dataset\samsumdata\pegasus-samsum-model")

In [40]:
## Save tokenizer
tokenizer.save_pretrained(r"C:\Users\Owner\Downloads\github\text-summarizer\dataset\samsumdata\tokenizer")

('C:\\Users\\Owner\\Downloads\\github\\text-summarizer\\dataset\\samsumdata\\tokenizer\\tokenizer_config.json',
 'C:\\Users\\Owner\\Downloads\\github\\text-summarizer\\dataset\\samsumdata\\tokenizer\\special_tokens_map.json',
 'C:\\Users\\Owner\\Downloads\\github\\text-summarizer\\dataset\\samsumdata\\tokenizer\\spiece.model',
 'C:\\Users\\Owner\\Downloads\\github\\text-summarizer\\dataset\\samsumdata\\tokenizer\\added_tokens.json',
 'C:\\Users\\Owner\\Downloads\\github\\text-summarizer\\dataset\\samsumdata\\tokenizer\\tokenizer.json')

In [41]:
#Load

tokenizer = AutoTokenizer.from_pretrained(r"C:\Users\Owner\Downloads\github\text-summarizer\dataset\samsumdata\tokenizer")

In [43]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}



sample_text = dataset_samsum["test"][0]["dialogue"]

reference = dataset_samsum["test"][0]["summary"]

pipe = pipeline("summarization", model=r"C:\Users\Owner\Downloads\github\text-summarizer\dataset\samsumdata\pegasus-samsum-model",tokenizer=tokenizer)

## 
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda can't find Betty's number. She asks Larry for her number. Hannah suggests she text Larry instead. Amanda will send Betty a message.
